In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mlt

In [ ]:
players=pd.read_csv('/kaggle/input/epl-stats-20192020/players_1920.csv')
players.head()

In [ ]:
a=players['full'].unique
players.insert(column='name',value=a,loc=1)

In [ ]:
players['team'].unique()

**GETTING ALL PLAYERS FROM A SPECIFIC TEAM**

In [ ]:
def team(x):
    return players[players['team']==x][['full','value','goals_scored','minutes']]

team('Man City')

In [ ]:
epl2020=pd.read_csv('/kaggle/input/epl-stats-20192020/epl2020.csv')
epl2020.head()

In [ ]:
players.shape

In [ ]:
epl2020.shape

**MAKING THE CURRENT TABLE**

In [ ]:
 epl2020.columns

In [ ]:
players.columns

In [ ]:
POSITIONS = np.array(range(1, 21))

In [ ]:
teams=epl2020['teamId'].unique()
team_results=[]
for team in teams:
    
    team_data = epl2020[epl2020['teamId'] == team]

    wins = team_data['wins'].sum()
    draws = team_data['draws'].sum()
    losses = team_data['loses'].sum()
    scored = team_data['scored'].sum()
    conceded = team_data['missed'].sum()
    games = wins + draws + losses
    points = (3 * wins) + draws
    goal_difference = scored - conceded
    
    team_results.append([team, games, wins, draws, losses, scored, conceded, goal_difference, points])

current_table=pd.DataFrame(team_results , columns=['Team', 'P', 'W', 'D', 'L', 'F', 'A', 'GD', 'Points'])
current_table.sort_values(by=['Points','GD','F'],ascending=False,inplace=True,ignore_index=True)
current_table.set_index(POSITIONS,inplace=True)
current_table.head(20)

**TABLE2:POSITIONS ON EXPECTED GOALS**

In [ ]:
#xG=expected goals scored xGA=expected goals conceded
epl2020['goals_scored']=epl2020['xG'].apply(lambda x:round(x))
epl2020['goals_conceded']=epl2020['xGA'].apply(lambda x:round(x))
epl2020['xwin']=epl2020.apply(lambda x:1 if x['goals_scored']>x['goals_conceded'] else 0,axis=1)
epl2020['xdraw']=epl2020.apply(lambda x:1 if x['goals_scored']==x['goals_conceded'] else 0,axis=1)
epl2020['xloss']=epl2020.apply(lambda x:1 if x['goals_scored']<x['goals_conceded'] else 0,axis=1)

In [ ]:
x_team_results=[]
for team in teams:
    x_team_data=epl2020[epl2020['teamId']==team]
    
    wins = x_team_data['xwin'].sum()
    draws = x_team_data['xdraw'].sum()
    losses = x_team_data['xloss'].sum()
    scored = x_team_data['goals_scored'].sum()
    conceded = x_team_data['goals_conceded'].sum()
    games = wins + draws + losses
    points = (3 * wins) + draws
    goal_difference = scored - conceded
    
    x_team_results.append([team, games, wins, draws, losses, scored, conceded, goal_difference, points])

x_table=pd.DataFrame(x_team_results , columns=['Team', 'P', 'W', 'D', 'L', 'F', 'A', 'GD', 'Points'])
x_table.sort_values(by=['Points','GD','F'],ascending=False,inplace=True,ignore_index=True)
x_table.set_index(POSITIONS,inplace=True)
x_table.head(20)

**COMPARING THE EXPECTED AND ORIGINAL TABLE**

In [ ]:
team_positions=[]
for team in teams:
    current_pos=current_table[current_table['Team']==team].index[0]
    x_pos=x_table[x_table['Team']==team].index[0]
    overperforming ='Yes' if current_pos<x_pos else 'No'
    team_positions.append([team,current_pos,x_pos,overperforming])

table3=pd.DataFrame(team_positions,columns=['Team','Current Position','Expected Position','Overperforming'])
table3.sort_values(by='Current Position',ascending=True,inplace=True,ignore_index=True)
table3.head(20)

In [ ]:
import matplotlib.pyplot as plt

**EVALUATING PERFORMANCE ON BASED OF LAST 6 GAMES**

In [ ]:
recent_form = []

for team in teams:
    team_data = epl2020[epl2020['teamId'] == team].tail(6)
    
    wins = team_data['wins'].sum()
    draws = team_data['draws'].sum()
    points = (3 * wins) + draws
    
    recent_form.append([team, points])

recent_form.sort(key=lambda x: x[1])

plt.figure(figsize = (8, 8))
plt.barh(range(20), [x[1] for x in recent_form])
plt.xlabel('Points')
plt.ylabel('Team')
plt.title('Points In Last 6 Games')
plt.yticks(range(20), [x[0] for x in recent_form])
plt.show()

**PREDICTING FINAL POSITIONS**

In [ ]:
team_points = []
for team in teams:
    points_per_game = [x for x in recent_form if x[0] == team][0][1] / 6
    team_data = current_table[current_table['Team'] == team].iloc[0]
    games_to_play = 38 - team_data['P']
    new_points = int(team_data['Points'] + round(points_per_game * games_to_play))
    team_points.append([team, new_points])

predicted_table = pd.DataFrame(team_points, columns=['Team', 'Points'])
predicted_table.sort_values(by=['Points'], ascending=False, inplace=True, ignore_index=True)
predicted_table.set_index(POSITIONS, inplace=True)
predicted_table.head(20)

**EDA ON PLAYER DATA**

In [ ]:
players.info()
players.columns

In [ ]:
players.describe()

In [ ]:
import seaborn as sn

In [ ]:
#cummulative sum of goals scored per player
players['Total goals']=players.sort_values(by=['round']).groupby(['full'])['goals_scored'].cumsum()

In [ ]:
#top5 scorers
top_scorers=players.groupby(['full'])['Total goals'].max().sort_values(ascending=False).head(5).index.get_level_values('full').to_list()
top = players.full.isin(top_scorers)
scorers =players[top]

In [ ]:
plt.figure(figsize=(10, 10))
sn.set(style='whitegrid', rc={"grid.linewidth": 0.1})
sn.set_context("paper", font_scale=0.9)
plt.xlim(1,29)
sn.lineplot('round', 'Total goals', data=scorers, hue='full',size = 'full').set_title("Top 5 - Goals")

In [ ]:
#top 5 players on basis of assists
players['Total assists']=players.sort_values(by=['round']).groupby(['full'])['assists'].cumsum()
top_assist=players.groupby(['full'])['Total assists'].max().sort_values(ascending=False).head(5).index.get_level_values('full').to_list()
top = players.full.isin(top_assist)
scorers =players[top]
plt.figure(figsize=(10, 10))
sn.set(style='whitegrid', rc={"grid.linewidth": 0.1})
sn.set_context("paper", font_scale=0.9)
plt.xlim(1,29)
sn.lineplot('round', 'Total assists', data=scorers, hue='full',size = 'full').set_title("Top 5 - Assists")